<a href="https://colab.research.google.com/github/L0521/hufs-logic-programming/blob/main/%ED%98%B8%ED%85%94_%EC%B7%A8%EC%86%8C_%EC%98%88%EC%83%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#202302834 이승준
#ValueError 해결 못함
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

#깃허브에 준비된 데이터 가져오기
Url_hotel_cancel = "https://raw.githubusercontent.com/L0521/hufs-logic-programming/main/Hotel%20Reservations.csv"

#호텔 취소 관련 데이터 불러오기
df = pd.read_csv(Url_hotel_cancel)

#자료 형태 보기
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

In [ ]:
#범주형 열의 객체 값을 의미 있는 숫자로 변환
encoder = LabelEncoder()
df[['type_of_meal_plan','room_type_reserved',
      'market_segment_type']] = df[[
          'type_of_meal_plan','room_type_reserved','market_segment_type',]].apply(encoder.fit_transform)

mapping = {'Not_Canceled':0,'Canceled':1}
df['booking_status'] = df['booking_status'].map(mapping)

df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,0,0,0,224,2017,10,2,3,0,0,0,65.00,0,0
1,INN00002,2,0,2,3,3,0,0,5,2018,11,6,4,0,0,0,106.68,1,0
2,INN00003,1,0,2,1,0,0,0,1,2018,2,28,4,0,0,0,60.00,0,1
3,INN00004,2,0,0,2,0,0,0,211,2018,5,20,4,0,0,0,100.00,0,1
4,INN00005,2,0,1,1,3,0,0,48,2018,4,11,4,0,0,0,94.50,0,1


In [ ]:
# 판다스에서 제공하는 get_dummies 함수를 사용하여 원-핫 인코딩 수행
df = pd.get_dummies(df, columns=['room_type_reserved'])

# 결과 확인
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,lead_time,arrival_year,arrival_month,...,avg_price_per_room,no_of_special_requests,booking_status,room_type_reserved_0,room_type_reserved_1,room_type_reserved_2,room_type_reserved_3,room_type_reserved_4,room_type_reserved_5,room_type_reserved_6
0,INN00001,2,0,1,2,0,0,224,2017,10,...,65.00,0,0,1,0,0,0,0,0,0
1,INN00002,2,0,2,3,3,0,5,2018,11,...,106.68,1,0,1,0,0,0,0,0,0
2,INN00003,1,0,2,1,0,0,1,2018,2,...,60.00,0,1,1,0,0,0,0,0,0
3,INN00004,2,0,0,2,0,0,211,2018,5,...,100.00,0,1,1,0,0,0,0,0,0
4,INN00005,2,0,1,1,3,0,48,2018,4,...,94.50,0,1,1,0,0,0,0,0,0


In [ ]:
# 여러 범주형 열에 대해 일괄적으로 원-핫 인코딩
df = pd.get_dummies(df, columns=['room_type_reserved_0', 'type_of_meal_plan', 'market_segment_type'])

# 결과 확인
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,...,room_type_reserved_0_1,type_of_meal_plan_0,type_of_meal_plan_1,type_of_meal_plan_2,type_of_meal_plan_3,market_segment_type_0,market_segment_type_1,market_segment_type_2,market_segment_type_3,market_segment_type_4
0,INN00001,2,0,1,2,0,224,2017,10,2,...,1,1,0,0,0,0,0,0,1,0
1,INN00002,2,0,2,3,0,5,2018,11,6,...,1,0,0,0,1,0,0,0,0,1
2,INN00003,1,0,2,1,0,1,2018,2,28,...,1,1,0,0,0,0,0,0,0,1
3,INN00004,2,0,0,2,0,211,2018,5,20,...,1,1,0,0,0,0,0,0,0,1
4,INN00005,2,0,1,1,0,48,2018,4,11,...,1,0,0,0,1,0,0,0,0,1


In [ ]:
#열의 인덱스 개수 확인
print(df.shape[1])

#열의 데이터프레임 정보 확인
print(df.info())

33
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   required_car_parking_space            36275 non-null  int64  
 6   lead_time                             36275 non-null  int64  
 7   arrival_year                          36275 non-null  int64  
 8   arrival_month                         36275 non-null  int64  
 9   arrival_date                          36275 non-null  int64  
 10  repeated_guest                        36275 non-null  int64  
 11  no_of_previo

In [ ]:
X = df.iloc[:,0:18]
y = df.iloc[:,18]

k = 5

In [ ]:
#KFold 함수 불러오기
kfold = KFold(n_splits=k, shuffle = True)

acc_score = []

In [ ]:
def model_fn():
  model = Sequential()
  model.add(Dense(24, input_dim=16, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
#k겹 교차 검증을 이용해 k번의 학습 실행
for train_index, test_index in kfold.split(X):
  X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # 1. 데이터 불러오기
  X_train_numeric = X_train.drop(['Booking_ID', 'booking_status'], axis=1)

  # 2. 형 변환
  X_train_numeric = X_train_numeric.astype('float32')
  y_train = y_train.astype('float32')

  # 3. 데이터 차원 변환
  X_train_numeric = np.array(X_train_numeric)

  model = model_fn()
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  history = model.fit(X_train_numeric, y_train, epochs=200, batch_size=10, verbose=0)

  accuracy = model.evaluate(X_test, y_test)[1]
  acc_score.append(accuracy)

ValueError: ignored

In [ ]:
#k번 실시된 정확도의 평균 구하기
avg_acc_score = sum(acc_score)/k

print('정확도: ',acc_score)
print('정확도 평균: ', avg_acc_score)

정확도:  []
정확도 평균:  0.0
